In [1]:
import pandas as pd

pd.set_option("max_rows", 60)

flights = pd.read_csv('csv/flights.csv', index_col='flight')
flights

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
flight,,,,,,,,,,,,,,,,,,
1545,2013,1,1,517,515,2,830,819,11,UA,N14228,EWR,IAH,227,1400,5,15,2013-01-01T05:00:00Z
1714,2013,1,1,533,529,4,850,830,20,UA,N24211,LGA,IAH,227,1416,5,29,2013-01-01T05:00:00Z
1141,2013,1,1,542,540,2,923,850,33,AA,N619AA,JFK,MIA,160,1089,5,40,2013-01-01T05:00:00Z
725,2013,1,1,544,545,-1,1004,1022,-18,B6,N804JB,JFK,BQN,183,1576,5,45,2013-01-01T05:00:00Z
461,2013,1,1,554,600,-6,812,837,-25,DL,N668DN,LGA,ATL,116,762,6,0,2013-01-01T06:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3393,2013,9,30,,1455,,,1634,,9E,,JFK,DCA,,213,14,55,2013-09-30T14:00:00Z
3525,2013,9,30,,2200,,,2312,,9E,,LGA,SYR,,198,22,0,2013-09-30T22:00:00Z
3461,2013,9,30,,1210,,,1330,,MQ,N535MQ,LGA,BNA,,764,12,10,2013-09-30T12:00:00Z


In [42]:
# On compte le nombre d'aéroports de destination
count_destinations_airport = flights.dest.nunique()
print(count_destinations_airport)

105


In [34]:
# On compte le nombre d'aéroports de départ
count_departures_airport = flights.origin.nunique()
print(count_departures_airport)

3


In [4]:
airports = pd.read_csv('csv/airports.csv', index_col='faa')
airlines = pd.read_csv('csv/airlines.csv', index_col='carrier')
planes = pd.read_csv('csv/planes.csv', index_col='tailnum')
weather = pd.read_csv('csv/weather.csv')
weather_with_3_doublons = pd.read_csv('csv/weather_with_3_doublons.csv')

airports

,name,lat,lon,alt,tz,dst,tzone
faa,,,,,,,
04G,Lansdowne Airport,41.130472,-80.619583,1044,-5,A,America/New_York
06A,Moton Field Municipal Airport,32.460572,-85.680028,264,-6,A,America/Chicago
06C,Schaumburg Regional,41.989341,-88.101243,801,-6,A,America/Chicago
06N,Randall Airport,41.431912,-74.391561,523,-5,A,America/New_York
09J,Jekyll Island Airport,31.074472,-81.427778,11,-5,A,America/New_York
...,...,...,...,...,...,...,...
ZUN,Black Rock,35.083228,-108.791778,6454,-7,A,America/Denver
ZVE,New Haven Rail Station,41.298669,-72.925992,7,-5,A,America/New_York
ZWI,Wilmington Amtrak Station,39.736667,-75.551667,0,-5,A,America/New_York


In [40]:
# 1) Nombre d'aéroports
total_count_airports = airports.index.nunique()
print(total_count_airports)

1458


In [37]:
# Aéroports qui ne passent pas à l'heure d'été (= no dst)
no_dst_airports = airports['dst'].value_counts()['N']
print(f'Airports which dont use summer time {no_dst_airports}')

# nombre de timezones
nb_timezones = airports.tzone.nunique()
print(f'Number of timzones {nb_timezones}')

Airports which dont use summer time 23
Number of timzones 10


In [8]:
airlines

,name
carrier,
9E,Endeavor Air Inc.
AA,American Airlines Inc.
AS,Alaska Airlines Inc.
B6,JetBlue Airways
DL,Delta Air Lines Inc.
EV,ExpressJet Airlines Inc.
F9,Frontier Airlines Inc.
FL,AirTran Airways Corporation
HA,Hawaiian Airlines Inc.


In [44]:
# nombre de compagnies
airlines_count = len(airlines.index)
print(airlines_count)

16


In [10]:
planes

,year,type,manufacturer,model,engines,seats,speed,engine
tailnum,,,,,,,,
N10156,2004,Fixed wing multi engine,EMBRAER,EMB-145XR,2,55,,Turbo-fan
N10156,1998,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,,Turbo-fan
N103US,1999,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,,Turbo-fan
N104UW,1999,Fixed wing multi engine,AIRBUS INDUSTRIE,A320-214,2,182,,Turbo-fan
N10575,2002,Fixed wing multi engine,EMBRAER,EMB-145LR,2,55,,Turbo-fan
...,...,...,...,...,...,...,...,...
N997AT,2002,Fixed wing multi engine,BOEING,717-200,2,100,,Turbo-fan
N997DL,1992,Fixed wing multi engine,MCDONNELL DOUGLAS AIRCRAFT CO,MD-88,2,142,,Turbo-fan
N998AT,2002,Fixed wing multi engine,BOEING,717-200,2,100,,Turbo-fan


In [39]:
# nombre d'avions
planes_count = len(planes.index)
print(planes_count)

3322


In [19]:
flights

,year,month,day,dep_time,sched_dep_time,dep_delay,arr_time,sched_arr_time,arr_delay,carrier,tailnum,origin,dest,air_time,distance,hour,minute,time_hour
flight,,,,,,,,,,,,,,,,,,
1545,2013,1,1,517,515,2,830,819,11,UA,N14228,EWR,IAH,227,1400,5,15,2013-01-01T05:00:00Z
1714,2013,1,1,533,529,4,850,830,20,UA,N24211,LGA,IAH,227,1416,5,29,2013-01-01T05:00:00Z
1141,2013,1,1,542,540,2,923,850,33,AA,N619AA,JFK,MIA,160,1089,5,40,2013-01-01T05:00:00Z
725,2013,1,1,544,545,-1,1004,1022,-18,B6,N804JB,JFK,BQN,183,1576,5,45,2013-01-01T05:00:00Z
461,2013,1,1,554,600,-6,812,837,-25,DL,N668DN,LGA,ATL,116,762,6,0,2013-01-01T06:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3393,2013,9,30,,1455,,,1634,,9E,,JFK,DCA,,213,14,55,2013-09-30T14:00:00Z
3525,2013,9,30,,2200,,,2312,,9E,,LGA,SYR,,198,22,0,2013-09-30T22:00:00Z
3461,2013,9,30,,1210,,,1330,,MQ,N535MQ,LGA,BNA,,764,12,10,2013-09-30T12:00:00Z


## Nombre de vols annulés

In [46]:
len(flights[flights['dep_time'] == ' '])

8255

## Aeroports de départ le plus empreinté 

In [55]:
flights['origin'].value_counts(ascending=False).head(1)

EWR    120835
Name: origin, dtype: int64

In [99]:
dests = flights['dest'].value_counts(ascending = False).head(10)
dests

ORD    17283
ATL    17215
LAX    16174
BOS    15508
MCO    14082
CLT    14064
SFO    13331
FLL    12055
MIA    11728
DCA     9705
Name: dest, dtype: int64

In [101]:
dests = dests.reset_index()
dests.columns =['faa' , 'dest']

In [134]:
airports_most_used = pd.merge(dests, airports[['name']], how='left', on='faa')
airports_less_used = airports_most_used.iloc[::-1]
airports_less_used

,faa,dest,name
9,DCA,9705,Ronald Reagan Washington Natl
8,MIA,11728,Miami Intl
7,FLL,12055,Fort Lauderdale Hollywood Intl
6,SFO,13331,San Francisco Intl
5,CLT,14064,Charlotte Douglas Intl
4,MCO,14082,Orlando Intl
3,BOS,15508,General Edward Lawrence Logan Intl
2,LAX,16174,Los Angeles Intl
1,ATL,17215,Hartsfield Jackson Atlanta Intl
0,ORD,17283,Chicago Ohare Intl


In [133]:
airports_most_used['Pourcentage (%)'] = (airports_most_used["dest"]/flights["dest"].value_counts().sum())* 100
airports_most_used

,faa,dest,name,Pourcentage (%)
0,ORD,17283,Chicago Ohare Intl,5.131898
1,ATL,17215,Hartsfield Jackson Atlanta Intl,5.111706
2,LAX,16174,Los Angeles Intl,4.802599
3,BOS,15508,General Edward Lawrence Logan Intl,4.604841
4,MCO,14082,Orlando Intl,4.181414
5,CLT,14064,Charlotte Douglas Intl,4.176070
6,SFO,13331,San Francisco Intl,3.958417
7,FLL,12055,Fort Lauderdale Hollywood Intl,3.579531
8,MIA,11728,Miami Intl,3.482433
9,DCA,9705,Ronald Reagan Washington Natl,2.881737


In [15]:
weather

,origin,year,month,day,hour,temp,dewp,humid,wind_dir,wind_speed,wind_gust,precip,pressure,visib,time_hour
0,EWR,2013,1,1,0,37.04,21.92,53.97,230,10.357019999999999,11.918651475599997,0.0,1013.9,10.0,2013-01-01T00:00:00Z
1,EWR,2013,1,1,1,37.04,21.92,53.97,230,13.809359999999998,15.891535300799996,0.0,1013,10.0,2013-01-01T01:00:00Z
2,EWR,2013,1,1,2,37.94,21.92,52.09,230,12.658579999999999,14.567240692399997,0.0,1012.6,10.0,2013-01-01T02:00:00Z
3,EWR,2013,1,1,3,37.94,23,54.51,230,13.809359999999998,15.891535300799996,0.0,1012.7,10.0,2013-01-01T03:00:00Z
4,EWR,2013,1,1,4,37.94,24.08,57.04,240,14.960139999999999,17.215829909199996,0.0,1012.8,10.0,2013-01-01T04:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26125,LGA,2013,12,30,19,35.96,19.94,51.78,340,13.809359999999998,15.891535300799996,0.0,1017.1,10.0,2013-12-30T19:00:00Z
26126,LGA,2013,12,30,20,33.98,17.06,49.51,330,17.261699999999998,19.864419125999994,0.0,1018.8,10.0,2013-12-30T20:00:00Z
26127,LGA,2013,12,30,21,32,15.08,49.19,340,14.960139999999999,17.215829909199996,0.0,1019.5,10.0,2013-12-30T21:00:00Z
26128,LGA,2013,12,30,22,30.92,12.92,46.74,320,17.261699999999998,19.864419125999994,0.0,1019.9,10.0,2013-12-30T22:00:00Z


In [16]:
weather_with_3_doublons

,origin,year,month,day,hour,temp,dewp,humid,wind_dir,wind_speed,wind_gust,precip,pressure,visib,time_hour
0,EWR,2013,1,1,1,39.02,26.06,59.37,270.0,10.35702,NaN,0.0,1012.0,10.0,2013-01-01T06:00:00Z
1,EWR,2013,1,1,2,39.02,26.96,61.63,250.0,8.05546,NaN,0.0,1012.3,10.0,2013-01-01T07:00:00Z
2,EWR,2013,1,1,3,39.02,28.04,64.43,240.0,11.50780,NaN,0.0,1012.5,10.0,2013-01-01T08:00:00Z
3,EWR,2013,1,1,4,39.92,28.04,62.21,250.0,12.65858,NaN,0.0,1012.2,10.0,2013-01-01T09:00:00Z
4,EWR,2013,1,1,5,39.02,28.04,64.43,260.0,12.65858,NaN,0.0,1011.9,10.0,2013-01-01T10:00:00Z
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
26110,LGA,2013,12,30,14,35.96,19.94,51.78,340.0,13.80936,21.86482,0.0,1017.1,10.0,2013-12-30T19:00:00Z
26111,LGA,2013,12,30,15,33.98,17.06,49.51,330.0,17.26170,21.86482,0.0,1018.8,10.0,2013-12-30T20:00:00Z
26112,LGA,2013,12,30,16,32.00,15.08,49.19,340.0,14.96014,23.01560,0.0,1019.5,10.0,2013-12-30T21:00:00Z
26113,LGA,2013,12,30,17,30.92,12.92,46.74,320.0,17.26170,NaN,0.0,1019.9,10.0,2013-12-30T22:00:00Z
